In [3]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver

In [247]:
"""
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, the tab will close automatically when it is done loading.
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&ref_=adv_prv/'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")

#func used to extract data by inputting the soup and the thing you want to find => "selector"
# def extract_data(soup, selector):
#     #tries to find selector, if unsuccessful will replace unfound data by None
#     try:
#         return soup.select(selector).text
#     except AttributeError:
#         print(AttributeError)
#         return None


while True:
    game_info = soup.find_all("div", class_="lister-item-content")
    
    for i in game_info:
        #gets game name
        try:
            game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
        except AttributeError:
            game_name = None
        print(game_name)
        
        #gets maturity rating
        try:
            game_maturity = i.find("span", class_="certificate").text.strip()
        except AttributeError:
            game_maturity = None
        print(f"Rating: {game_maturity}")

        #gets genre and split it into an array
        try:
            game_genre = i.find("span", class_="genre").text.strip()
            game_genre = list(game_genre.split(","))
        except AttributeError:
            game_genre = None
        print(game_genre)
        
        #get rating
        try:
            game_rating = i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip()
        except AttributeError:
            game_rating = None
        print(game_rating)
        
        #get Director & Stars and will split them
        try:
            #get both
            game_director_and_stars = i.find("p", class_="").text
            game_director, separator, game_stars = game_director_and_stars.partition("|")
            
            #split Director and put it in array
            deleted_text, separator, game_director = game_director.partition(":")
            game_director = game_director.strip()
            game_director = ''.join(game_director.splitlines())
            game_director = list(game_director.split(","))            
            
            #Stars
            deleted_text, separator, game_stars = game_stars.partition(":")
            game_stars = game_stars.strip()
            game_stars = ''.join(game_stars.splitlines())
            game_stars = list(game_stars.split(","))

        except AttributeError:
            game_director = None
        print(game_director)
        print(game_stars)
        
        
        print("-------------------------------------------------------------------------------")
    
    
    #find button to click on next page
    uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
    completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url
    
    print(completed_next_page_url)

    #replaces old url to completed_next_page_url
    url = completed_next_page_url
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    
#     html_request = requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")


driver.quit()

The Last of Us
Rating: M
['Action', ' Adventure', ' Drama']
9.7
['Neil Druckmann', ' Bruce Straley']
['Troy Baker', ' Ashley Johnson', ' Hana Hayes', ' Jeffrey Pierce']
-------------------------------------------------------------------------------
Hogwarts Legacy
Rating: T
['Action', ' Adventure', ' Fantasy']
9.2
['Natalie Gray', ' Nathan Hendrickson', ' Alan Tew']
['Alastair Duncan', ' Amy Louise Pemberton', ' Asif Ali', ' Christopher J James']
-------------------------------------------------------------------------------
The Last of Us: Part II
Rating: M
['Action', ' Adventure', ' Drama']
8.6
['Neil Druckmann', ' Kurt Margenau', ' Anthony Newman']
['Ashley Johnson', ' Troy Baker', ' Derek Phillips', ' Jeffrey Pierce']
-------------------------------------------------------------------------------
Atomic Heart
Rating: M
['Action', ' Adventure', ' Crime']
6.5
['Artyom Galeev']
['Alexander Lomov', ' Dmitriy Romashin', ' Maria Rudenko', ' Oleg Kursachev']
------------------------------

Fallout 4
Rating: M
['Action', ' Adventure', ' Sci-Fi']
8.6
['Todd Howard']
['Brian T. Delaney', ' Courtenay Taylor', ' Stephen Russell', ' Courtney Ford']
-------------------------------------------------------------------------------
Days Gone
Rating: M
['Action', ' Adventure', ' Drama']
8.3
['John Garvin', ' Jeff Ross']
['Sam Witwer', ' Jim Pirri', ' Courtnee Draper', ' Nishi Munshi']
-------------------------------------------------------------------------------
High on Life
Rating: M
['Action', ' Adventure', ' Comedy']
7.8
['Justin Roiland', ' J.B. Smoove', ' Betsy Sodaro', ' Tim Robinson']
['']
-------------------------------------------------------------------------------
Grand Theft Auto VI
Rating: M
['Action', ' Crime', ' Drama']
None
['Bryan Zampella']
['']
-------------------------------------------------------------------------------
Hi-Fi Rush
Rating: T
['Action', ' Music', ' Sci-Fi']
8.4
['John Johanas']
['Robbie Daymond', ' Erica Lindbeck', ' Sunil Malhotra', ' Gabe Kund

KeyboardInterrupt: 

In [23]:
"""
This will open an extra chrome tab just used to parse some of the data that I couldnt get normally from the website 
directly because it parsed back "None" Values for some elements even though it existed, therefor I couldn't get all the data 
I wanted to collect for this project. the chrome tab allows me read those without restrictions and it solves 
the "None" Value problem

IMPORTANT: Please dont close the tab or the project will break, the tab will close automatically when it is done loading.
"""
driver = webdriver.Chrome()
url = 'https://www.imdb.com/search/title/?title_type=video_game&ref_=adv_prv/'
# driver.maximize_window()
driver.get(url)
content = driver.page_source.encode('utf-8').strip()
""""""
soup = BeautifulSoup(content,"html.parser")




#func used to extract data by inputting the soup and the thing you want to find => "selector"
def extract_data(soup, selector, class_selector):
    try:
        return soup.find(selector, class_= class_selector).text.strip()
    except AttributeError:
#         print(AttributeError)
        return None


while True:
    game_info = soup.find_all("div", class_="lister-item-content")
    
    for i in game_info:
        #gets game name
        try:
            game_name = (i.find("h3", class_="lister-item-header")).a.text.strip()
        except AttributeError:
            game_name = None
        print(game_name)
        
        #gets maturity rating
        game_maturity = extract_data(i, "span", "certificate")
        print(f"Rating: {game_maturity}")

        #gets genre and split it into an array
        game_genre = extract_data(i, "span", "genre")
        game_genre = list(game_genre.split(","))
        print(f"Genre: {game_genre}")
        
        #get rating
        try:
            game_rating = i.find("div", class_="inline-block ratings-imdb-rating").strong.text.strip()
        except AttributeError:
            game_rating = None
        print(f"Rating: {game_rating}")
        
        #get Director & Stars and will split them
        #get both
        game_director_and_stars = extract_data(i, "p", "")
        game_director, separator, game_stars = game_director_and_stars.partition("|")
        
        #split Director and put it in array
        deleted_text, separator, game_director = game_director.partition(":")
        game_director = game_director.strip()
        game_director = ''.join(game_director.splitlines())
        game_director = list(game_director.split(","))
        
        #Stars
        deleted_text, separator, game_stars = game_stars.partition(":")
        game_stars = game_stars.strip()
        game_stars = ''.join(game_stars.splitlines())
        game_stars = list(game_stars.split(","))
        
        print(f"Director(s): {game_director}")
        print(f"Star(s): {game_stars}")
        
        #get votes
        game_votes = extract_data(i, "p", "sort-num_votes-visible")
        game_votes = ''.join(game_votes.splitlines())
        deleted_text, separator, game_votes = game_votes.partition(":")
#         game_votes = i.find("p", class_="sort-num_votes-visible").text.strip()
        print(f"Votes: {game_votes}")
        
        print("-------------------------------------------------------------------------------")
    
    
    #find button to click on next page
    uncompleted_next_page_url = soup.find("a", class_="lister-page-next next-page").get("href")
    completed_next_page_url = "https://www.imdb.com/" + uncompleted_next_page_url
    
    print(completed_next_page_url)

    #replaces old url to completed_next_page_url
    url = completed_next_page_url
    driver.get(url)
    content = driver.page_source.encode('utf-8').strip()
    
#     html_request = requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")


driver.quit()

The Last of Us
Rating: M
Genre: ['Action', ' Adventure', ' Drama']
Rating: 9.7
Director(s): ['Neil Druckmann', ' Bruce Straley']
Star(s): ['Troy Baker', ' Ashley Johnson', ' Hana Hayes', ' Jeffrey Pierce']
68,818
-------------------------------------------------------------------------------
The Last of Us: Part II
Rating: M
Genre: ['Action', ' Adventure', ' Drama']
Rating: 8.6
Director(s): ['Neil Druckmann', ' Kurt Margenau', ' Anthony Newman']
Star(s): ['Ashley Johnson', ' Troy Baker', ' Derek Phillips', ' Jeffrey Pierce']
36,093
-------------------------------------------------------------------------------
Hogwarts Legacy
Rating: T
Genre: ['Action', ' Adventure', ' Fantasy']
Rating: 9.1
Director(s): ['Natalie Gray', ' Nathan Hendrickson', ' Alan Tew']
Star(s): ['Alastair Duncan', ' Amy Louise Pemberton', ' Asif Ali', ' Carla Tassara']
2,830
-------------------------------------------------------------------------------
The Quarry
Rating: M
Genre: ['Fantasy', ' Horror', ' Thriller']

AttributeError: 'NoneType' object has no attribute 'splitlines'

In [240]:
#full
game_director_and_stars = soup.find("p", class_="").text
game_director, separator, game_stars = game_director_and_stars.partition("|")

print(game_director)
print(game_stars)


    Directors:
Neil Druckmann, 
Bruce Straley

 
    Stars:
Troy Baker, 
Ashley Johnson, 
Hana Hayes, 
Jeffrey Pierce



In [237]:
#Director
deleted_text, separator, game_director = game_director.partition(":")
game_director = game_director.strip()
game_director = ''.join(game_director.splitlines())
game_director = list(game_director.split(","))

print(game_director)

['Neil Druckmann', ' Bruce Straley']


In [241]:
#Stars
deleted_text, separator, game_stars = game_stars.partition(":")
game_stars = game_stars.strip()
game_stars = ''.join(game_stars.splitlines())
game_stars = list(game_stars.split(","))

print(game_stars)

['Troy Baker', ' Ashley Johnson', ' Hana Hayes', ' Jeffrey Pierce']
